In [1]:
# Download data fro3m http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# %lprun -f

In [2]:
import numpy as np
import re
import collections
import operator
from scipy.optimize import minimize
import copy

from scipy.optimize import fmin_bfgs

In [3]:
def findDic(text):
    words = re.compile('[a-zA-Z\']*[a-zA-Z]').findall(text)
    return collections.Counter(words)

In [4]:
def readData(directory, paths, limit=None):
    if limit:
        paths = paths[0:limit]

    data = []
    for path in paths:
        txt = open(directory + path).read()
        data.append(txt)
    return data
        
        
negDataPath = !ls data/neg/
negData = readData('./data/neg/', negDataPath, 1000)

posDataPath = !ls data/pos/
posData = readData('./data/pos/', posDataPath, 1000)

In [5]:
negData = [
    'I am exhausted of this work.',
    "I can't cooperate with this",
    'I do not enjoy my job',
    "I feel brilliant!",
    'Gary is a friend of mine.',
    'The date was good.'
]
posData = [
    'I love this burger.',
    'This is an brilliant place!',
    'I feel very good about these dates.',
    'This is my best work.',
    "What an awesome view"
]

test = [
    "I am not feeling well today.",
    "I can't believe I'm doing this.",
    'The date was good.',
    "I am not feeling well today.",
    "I can't believe I'm doing this.",
    'I do not enjoy my job'
]

In [6]:
print negData[0]
print "+++++"
print posData[0]

I am exhausted of this work.
+++++
I love this burger.


In [7]:
def findWordCount(data):
    wordCount = []
    for review in data:
        wordCount.append(findDic(review))
    return wordCount

negDataDic = findWordCount(negData)
posDataDic = findWordCount(posData)

In [8]:
def updateDic(mainDic, dicArray):
    mainDic = copy.deepcopy(mainDic)
    for dic in dicArray:
        mainDic.update(dic)
    
    # update indexes
    for index, key in enumerate(mainDic):
        mainDic[ key ] = index
        
    return mainDic

allWords = {}
allWords = updateDic(allWords, negDataDic)
allWords = updateDic(allWords, posDataDic)

In [9]:
def findRow(words, text, boolean=False):
    row = np.zeros(len(allWords))
    wordsDic = findDic(text)
    for word in words:
        if word in allWords:
            row[allWords[word]] = 1 if boolean else wordsDic[word]
    return row

In [10]:
def createTable(allWords, negData, y0, posData, y1):
    x = []
    for data in negData + posData:
        x.append( findRow(allWords, data) )
    x = np.row_stack(x)
    
    y = np.append(y0 * np.ones(len(negData)), y1 * np.ones(len(posData)))
    return x, y

In [11]:
x, y = createTable(allWords, negData, -100, posData, 100)

In [39]:
def maxFunc(a, x, y):
    y1 = np.repeat(x, len(x)).reshape((len(x), len(x)))
    
    return y1

In [ ]:
x = np.array([1,2,3,4,5,6])
maxFunc()

In [42]:
x = np.array([[1,2,3,4], [5,6,7,8], [9,0,1,2], [3,4,5,6], [7,8,9,0]])

In [74]:
x = np.array([[1,2],[3,4]])

In [76]:
xx = np.array([i for i in x for _ in range(len(x))]).reshape(4,5,5)

ValueError: cannot reshape array of size 8 into shape (4,5,5)

In [73]:
xx

array([[[1, 2, 3, 4, 1],
        [2, 3, 4, 1, 2],
        [3, 4, 1, 2, 3],
        [4, 1, 2, 3, 4],
        [5, 6, 7, 8, 5]],

       [[6, 7, 8, 5, 6],
        [7, 8, 5, 6, 7],
        [8, 5, 6, 7, 8],
        [9, 0, 1, 2, 9],
        [0, 1, 2, 9, 0]],

       [[1, 2, 9, 0, 1],
        [2, 9, 0, 1, 2],
        [3, 4, 5, 6, 3],
        [4, 5, 6, 3, 4],
        [5, 6, 3, 4, 5]],

       [[6, 3, 4, 5, 6],
        [7, 8, 9, 0, 7],
        [8, 9, 0, 7, 8],
        [9, 0, 7, 8, 9],
        [0, 7, 8, 9, 0]]])